In [2]:
!wget -O diabetes_train.csv.gz https://github.com/scikit-learn/scikit-learn/raw/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_data_raw.csv.gz
!wget -O diabetes_target.csv.gz https://github.com/scikit-learn/scikit-learn/raw/98cf537f5c538fdbc9d27b851cf03ce7611b8a48/sklearn/datasets/data/diabetes_target.csv.gz
!gzip -d diabetes_train.csv.gz
!gzip -d diabetes_target.csv.gz

In [3]:
from bastionlab import Connection
import polars as pl

connection = Connection("localhost")

In [4]:
# load dataset
train_df = pl.read_csv(
    "diabetes_train.csv",
    sep=" ",
    has_header=False,
    new_columns=["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
)
target_df = pl.read_csv(
    "diabetes_target.csv",
    has_header=False,
    new_columns=["target"],
)

In [5]:
from bastionlab.polars.policy import Policy, TrueRule, Review, Aggregation

policy = Policy(
    safe_zone=Aggregation(min_agg_size=20), unsafe_handling=Review(), savable=False
)
train_rdf = connection.client.polars.send_df(train_df, policy)
target_rdf = connection.client.polars.send_df(target_df, policy)

/home/kwabena/base/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Perform linear regression on `diabetes` dataset.
from bastionlab.linfa.trainers import LinearRegression
from bastionlab.polars import train_test_split

train_rdf = train_rdf.select(pl.all().cast(pl.Float64)).collect()

train_array = train_rdf.to_array()
target_array = target_rdf.to_array()

train_X, test_X, train_Y, test_Y = train_test_split(
    train_array, target_array, test_size=0.4, shuffle=True
)

lr = LinearRegression()

lr.fit(train_X, train_Y)

y_pred = lr.predict(test_X)

print(y_pred.collect().fetch())

shape: (177, 1)
┌────────────┐
│ prediction │
│ ---        │
│ f64        │
╞════════════╡
│ 123.388614 │
├╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 182.618054 │
├╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 217.52973  │
├╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 136.053701 │
├╌╌╌╌╌╌╌╌╌╌╌╌┤
│ ...        │
├╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 142.901087 │
├╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 175.852744 │
├╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 94.228817  │
├╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 176.624805 │
└────────────┘
